In [81]:
import pymongo
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import itertools
import re
import string
from datetime import timedelta

In [82]:
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from gensim.models import doc2vec
from collections import namedtuple

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [83]:
client = pymongo.MongoClient('localhost', 27017)
db = client.csi

In [84]:
trainEvents = np.load('./TrainTestValCreation/trainEvents.npy')
testEvents = np.load('./TrainTestValCreation/testEvents.npy')
valEvents = np.load('./TrainTestValCreation/valEvents.npy')

## Building training set based on X_u (which was built using trainset userids)

In [85]:
global_userlist = np.load('./global_userlist.npy')
global_userlist = list(global_userlist)

In [86]:
X_u = np.load('./X_u/X_uSVD_trainset.npy')
partitionperEvent = np.load('./partitionperEventStats.npy')

In [7]:
numevents = len(trainEvents)
totalpartitions = max(partitionperEvent)

In [8]:
model = doc2vec.Doc2Vec.load('./doc2vecmodel')

In [75]:
partitions = 0
actualevent = 0
cnt = 0
inputperEvent = np.zeros((numevents, totalpartitions, 122))
#for e,l in trainEvents:
for e in [72]:
    tweetsperEvent = list(db.finalHydratedtweets.find({'myeventIndex': int(e)}).sort([("created_at" , 1)]))
    size = len(tweetsperEvent)
    if (size != 0):
        startTime =  (tweetsperEvent[0]["created_at"])
        print (startTime)
        stopTime =  (tweetsperEvent[size-1]["created_at"])
        print (stopTime)
        actualevent = actualevent + 1
        engagement = []
        partition = 0
        x_U = []
        k=0
        time_interval = 0
        time_intv = []
        sTime = startTime
        j = k
        while (sTime <= stopTime):
            #print (sTime)
            engagements = 0
            index = []
            eTime = sTime + timedelta(hours=1)
            #print ('k', j)
            for ev in tweetsperEvent[j:]:
                if (ev["created_at"] < eTime and ev["created_at"] >= sTime):
                    engagements = engagements + 1
                    ind = int(ev['user']['id_str'])
                    
                    index.append(X_u[global_userlist.index(ind)])
                    j = j+1 
                else:
                    break
            time_interval = time_interval + 1
            if(engagements != 0):
                partition = partition + 1
                engagement.append(engagements)
                x_U.append(np.mean(index, axis=0))
                time_intv.append(time_interval)
                time_interval = 0
            sTime = eTime
        print (partition)    
        for l in range(partition):
            inputperEvent[cnt, l, 0] = engagement[l]
            inputperEvent[cnt, l, 1] = time_intv[l]
            inputperEvent[cnt, l, 2:22] = x_U[l]
            inputperEvent[cnt, l, 22:122] = model.docvecs[l+partitions]
        partitions  = partitions + partition
        cnt = cnt + 1
        #evt = './FinalInputVectors/'+'e' + str(actualevent)
        #np.save(evt, inputperEvent)                   
    print (e, sum(engagement) == size)

#np.save('./Dataset1/TrainCaptureDataset', inputperEvent)

2015-11-04 18:33:35
2015-11-06 05:32:20
19
72 True


In [71]:
play = inputperEvent[0,:19,:]

In [72]:
play.shape

(19, 122)

In [76]:
play[:,1]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        3.,  1.,  6.,  3.,  7.,  2.])

## Generating valset 

In [87]:
numevents = len(valEvents)
totalpartitions = max(partitionperEvent)

In [88]:
partitions = 0
actualevent = 0
cnt = 0
inputperEvent = np.zeros((numevents, totalpartitions, 122))
for e,l in valEvents:
#for e in [72]:
    tweetsperEvent = list(db.finalHydratedtweets.find({'myeventIndex': int(e)}).sort([("created_at" , 1)]))
    size = len(tweetsperEvent)
    if (size != 0):
        startTime =  (tweetsperEvent[0]["created_at"])
        stopTime =  (tweetsperEvent[size-1]["created_at"])
        actualevent = actualevent + 1
        engagement = []
        partition = 0
        x_U = []
        k=0
        time_interval = 0
        time_intv = []
        sTime = startTime
        
        while (sTime <= stopTime):
            engagements = 0
            index = []
            eTime = sTime + timedelta(hours=1)
            j = k
            for ev in tweetsperEvent[j:]:
                if (ev["created_at"] < eTime and ev["created_at"] >= sTime):
                    engagements = engagements + 1
                    ind = int(ev['user']['id_str'])
                    if (ind in global_userlist):
                        index.append(X_u[global_userlist.index(ind)])
                    k = k+1 
                else:
                    break
            time_interval = time_interval + 1
            if(engagements != 0):
                
                partition = partition + 1
                engagement.append(engagements)
                if (len(index) == 0):
                    x_U.append(np.array(20*[0]))
                elif (len(index) ==1):
                    x_U.append(np.array(index))
                else:
                    x_U.append(np.mean(index, axis=0))
                time_intv.append(time_interval)
                time_interval = 0
            sTime = eTime
           
        for l in range(partition):
            inputperEvent[cnt, l, 0] = engagement[l]
            inputperEvent[cnt, l, 1] = time_intv[l]
            inputperEvent[cnt, l, 2:22] = x_U[l]
            tags = 'Event_'+str(e)+'_'+str(l+1)
            inputperEvent[cnt, l, 22:122] = model.docvecs[tags]
        #partitions  = partitions + partition
        cnt = cnt + 1
        #evt = './FinalInputVectors/'+'e' + str(actualevent)
        #np.save(evt, inputperEvent)                   
    print (e)
    
np.save('./Dataset1/ValCaptureDataset', inputperEvent)

763
720
942
945
82
645
470
65
427
685
587
612
737
595
50
445
179
585
314
738
700
433
297
617
241
249
62
217
934
415
593
178
611
878
666
299
770
66
51
377
691
140
526
698
530
362
73
509
750
209
653


In [80]:
inputperEvent[0,:19,0]

array([ 5.,  3.,  2.,  3.,  3.,  4.,  6.,  3.,  1.,  2.,  2.,  3.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.])

In [137]:
np.sum(inputperEvent)

116436.544592071

In [138]:
X_u[global_userlist.index(23863802)]

array([ -4.73721098e-03,  -1.92332453e-03,  -3.76522775e-03,
        -8.88505586e-04,  -1.68959319e-03,   1.00627407e-04,
         1.79387087e-05,  -2.53060619e-03,  -1.00170662e-04,
         8.93910815e-03,   1.20213154e-02,   8.40636115e-02,
         8.25797938e-01,  -2.85368136e-03,   4.84504945e-03,
        -3.12585400e-04,   1.88613851e-03,   1.28509394e-03,
        -3.17723760e-05,  -5.02770134e-04])

## Generating test set

In [139]:
numevents = len(testEvents)
totalpartitions = max(partitionperEvent)

In [140]:
partitions = 0
actualevent = 0
cnt = 0
inputperEvent = np.zeros((numevents, totalpartitions, 122))
for e,l in testEvents:
    tweetsperEvent = list(db.finalHydratedtweets.find({'myeventIndex': int(e)}).sort([("created_at" , 1)]))
    size = len(tweetsperEvent)
    if (size != 0):
        startTime =  (tweetsperEvent[0]["created_at"])
        stopTime =  (tweetsperEvent[size-1]["created_at"])
        actualevent = actualevent + 1
        engagement = []
        partition = 0
        x_U = []
        k=0
        time_interval = 0
        time_intv = []
        sTime = startTime
        
        while (sTime <= stopTime):
            engagements = 0
            index = []
            eTime = sTime + timedelta(hours=1)
            j = k
            for ev in tweetsperEvent[j:]:
                if (ev["created_at"] < eTime and ev["created_at"] >= sTime):
                    engagements = engagements + 1
                    ind = int(ev['user']['id_str'])
                    if (ind in global_userlist):
                        index.append(X_u[global_userlist.index(ind)])
                    k = k+1 
                else:
                    break
            time_interval = time_interval + 1
            if(engagements != 0):
                
                partition = partition + 1
                engagement.append(engagements)
                if (len(index) == 0):
                    x_U.append(np.array(20*[0]))
                elif (len(index) ==1):
                    x_U.append(np.array(index))
                else:
                    x_U.append(np.mean(index, axis=0))
                time_intv.append(time_interval)
                time_interval = 0
            sTime = eTime
           
        for l in range(partition):
            inputperEvent[cnt, l, 0] = engagement[l]
            inputperEvent[cnt, l, 1] = time_intv[l]
            inputperEvent[cnt, l, 2:22] = x_U[l]
            tags = 'Event_'+str(e)+'_'+str(l+1)
            inputperEvent[cnt, l, 22:122] = model.docvecs[tags]
        #partitions  = partitions + partition
        cnt = cnt + 1
        #evt = './FinalInputVectors/'+'e' + str(actualevent)
        #np.save(evt, inputperEvent)                   
    print (e)
    
np.save('./Dataset1/TestCaptureDataset', inputperEvent)

840
923
776
619
854
872
833
401
842
627
492
194
251
801
310
641
96
341
924
296
571
659
901
665
946
699
538
245
518
803
759
384
964
322
765
34
247
703
126
458
95
677
970
580
618
11
351
887
227
647
881
830
654
784
643
199
813
170
949
760
333
600
826
606
642
212
123
432
968
578
857
246
319
860
673
902
809
99
302
290
407
834
442
726
431
358
486
20
793
938
675
583
695
984
190
295
722
511
678
316
214
428
330
936
757
383
83
18
307
822
958
391
493
426
762
402
485
566
110
90
620
270
915
868
629
293
91
773
244
349
575
135
885
425
321
158
273
77
188
513
287
89
907
125
100
733
570
693


In [50]:
inputperEvent.shape

(148, 9189, 122)

In [52]:
np.sum(inputperEvent)

0.0